In [1]:
%load_ext autoreload
%autoreload 2



import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm
from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
SAVE_PATH = "./data/"
train_data = np.load(SAVE_PATH + 'train_inputs.npy')
val_data = np.load(SAVE_PATH + 'val_inputs.npy')
test_data = np.load(SAVE_PATH + 'test_inputs.npy')
train_labels = np.load(SAVE_PATH + 'train_labels.npy')
val_labels = np.load(SAVE_PATH + 'val_labels.npy')
test_labels = np.load(SAVE_PATH + 'test_labels.npy')

In [3]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(val_labels.shape)
print(test_labels.shape)


(93328, 3920)
(10980, 3920)
(12106, 3920)
(93328,)
(10980,)
(12106,)


In [4]:
Y_train_hot = to_categorical(train_labels)
Y_val_hot = to_categorical(val_labels)
Y_test_hot = to_categorical(test_labels)

In [5]:
print(Y_train_hot.shape)
print(Y_val_hot.shape)
print(Y_test_hot.shape)

(93328, 37)
(10980, 37)
(12106, 37)


In [6]:
#{'desired_samples': 16000, 'window_size_samples': 480, 'window_stride_samples': 160, 'spectrogram_length': 98, 'fingerprint_width': 40, 'fingerprint_size': 3920, 'label_count': 37, 'sample_rate': 16000, 'preprocess': 'mfcc', 'average_window_width': -1}
input_time_size = 98
fingerprint_width = 40
feature_dim_1 = input_time_size
feature_dim_2 = fingerprint_width
channel = 1
batch_size = 100
epochs = 1000
verbose = 1

In [7]:
40*98

3920

In [8]:
X_train = train_data.reshape(train_data.shape[0], feature_dim_1, feature_dim_2, channel)
X_val = val_data.reshape(val_data.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = test_data.reshape(test_data.shape[0], feature_dim_1, feature_dim_2, channel)

### model definition

In [9]:
# make the same model as the tensorflow model and test if it works.
# then may be we can replicate the same


def get_model_cnn(num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [11]:
# training a model on urban Noise data
model = get_model_cnn(37)
model.summary()
# checkpoint
filepath= "./models/model4.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(X_train, Y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, 
                    validation_data=(X_val, Y_val_hot), callbacks=callbacks_list )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 97, 39, 32)        160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 38, 48)        6192      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 95, 37, 120)       23160     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 18, 120)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 47, 18, 120)       0         
_________________________________________________________________
conv2d_4 (Conv2D)    


Epoch 00053: val_acc did not improve from 0.89144
Epoch 54/1000
93328/93328 [==============================] - 92s 985us/step - loss: 0.3112 - acc: 0.9148 - val_loss: 0.6284 - val_acc: 0.8801

Epoch 00054: val_acc did not improve from 0.89144
Epoch 55/1000
93328/93328 [==============================] - 92s 989us/step - loss: 0.3068 - acc: 0.9172 - val_loss: 0.6640 - val_acc: 0.8843

Epoch 00055: val_acc did not improve from 0.89144
Epoch 56/1000
93328/93328 [==============================] - 92s 990us/step - loss: 0.2998 - acc: 0.9179 - val_loss: 0.8196 - val_acc: 0.8830

Epoch 00056: val_acc did not improve from 0.89144
Epoch 57/1000
93328/93328 [==============================] - 92s 990us/step - loss: 0.3082 - acc: 0.9171 - val_loss: 0.5836 - val_acc: 0.8848

Epoch 00057: val_acc did not improve from 0.89144
Epoch 58/1000
93328/93328 [==============================] - 94s 1ms/step - loss: 0.2962 - acc: 0.9192 - val_loss: 0.9791 - val_acc: 0.8773

Epoch 00058: val_acc did not improve

93328/93328 [==============================] - 91s 980us/step - loss: 0.2850 - acc: 0.9271 - val_loss: 0.6435 - val_acc: 0.8406

Epoch 00096: val_acc did not improve from 0.89217
Epoch 97/1000
93328/93328 [==============================] - 91s 980us/step - loss: 0.2857 - acc: 0.9261 - val_loss: 0.5622 - val_acc: 0.8813

Epoch 00097: val_acc did not improve from 0.89217
Epoch 98/1000
93328/93328 [==============================] - 91s 980us/step - loss: 0.2862 - acc: 0.9272 - val_loss: 0.6531 - val_acc: 0.8740

Epoch 00098: val_acc did not improve from 0.89217
Epoch 99/1000
93328/93328 [==============================] - 91s 980us/step - loss: 0.2850 - acc: 0.9263 - val_loss: 0.6024 - val_acc: 0.8874

Epoch 00099: val_acc did not improve from 0.89217
Epoch 100/1000
93328/93328 [==============================] - 91s 979us/step - loss: 0.2874 - acc: 0.9267 - val_loss: 0.6361 - val_acc: 0.8887

Epoch 00100: val_acc did not improve from 0.89217
Epoch 101/1000
93328/93328 [====================


Epoch 00137: val_acc did not improve from 0.89235
Epoch 138/1000
93328/93328 [==============================] - 90s 965us/step - loss: 0.2866 - acc: 0.9289 - val_loss: 0.6509 - val_acc: 0.8912

Epoch 00138: val_acc did not improve from 0.89235
Epoch 139/1000
93328/93328 [==============================] - 90s 965us/step - loss: 0.2832 - acc: 0.9284 - val_loss: 0.5829 - val_acc: 0.8839

Epoch 00139: val_acc did not improve from 0.89235
Epoch 140/1000
93328/93328 [==============================] - 90s 966us/step - loss: 0.2903 - acc: 0.9281 - val_loss: 0.6792 - val_acc: 0.8831

Epoch 00140: val_acc did not improve from 0.89235
Epoch 141/1000
93328/93328 [==============================] - 90s 965us/step - loss: 0.2818 - acc: 0.9300 - val_loss: 0.6475 - val_acc: 0.8825

Epoch 00141: val_acc did not improve from 0.89235
Epoch 142/1000
93328/93328 [==============================] - 90s 965us/step - loss: 0.2842 - acc: 0.9293 - val_loss: 0.6487 - val_acc: 0.8742

Epoch 00142: val_acc did not 


Epoch 00178: val_acc did not improve from 0.89426
Epoch 179/1000
93328/93328 [==============================] - 90s 964us/step - loss: 0.2962 - acc: 0.9288 - val_loss: 0.6212 - val_acc: 0.8750

Epoch 00179: val_acc did not improve from 0.89426
Epoch 180/1000
93328/93328 [==============================] - 90s 963us/step - loss: 0.2953 - acc: 0.9286 - val_loss: 0.6168 - val_acc: 0.8614

Epoch 00180: val_acc did not improve from 0.89426
Epoch 181/1000
93328/93328 [==============================] - 90s 963us/step - loss: 0.2892 - acc: 0.9301 - val_loss: 0.6787 - val_acc: 0.8861

Epoch 00181: val_acc did not improve from 0.89426
Epoch 182/1000
93328/93328 [==============================] - 90s 964us/step - loss: 0.2916 - acc: 0.9291 - val_loss: 0.6540 - val_acc: 0.8873

Epoch 00182: val_acc did not improve from 0.89426
Epoch 183/1000
93328/93328 [==============================] - 90s 963us/step - loss: 0.2844 - acc: 0.9304 - val_loss: 0.6672 - val_acc: 0.8934

Epoch 00183: val_acc did not 

KeyboardInterrupt: 

In [12]:
# loading the model with the best val accuracy
model_test_un = get_model_cnn(37)
weights_path = TEST_PATH "./models/model4.hdf5"
model_test_un.load_weights(weights_path)
model_test_un.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 97, 39, 32)        160       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 96, 38, 48)        6192      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 95, 37, 120)       23160     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 47, 18, 120)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 47, 18, 120)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 46, 17, 120)       57720     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 45, 16, 120)       57720     
__________

In [14]:
#train and test loss and scores respectively
train_loss_score=model_test_un.evaluate(X_train,Y_train_hot)
val_loss_score=model_test_un.evaluate(X_val,Y_val_hot)
test_loss_score=model_test_un.evaluate(X_test,Y_test_hot)
print(train_loss_score)
print(val_loss_score)
print(test_loss_score)

12106/12106 [==============================] - 5s 381us/step
[0.01215071379404751, 0.9971712669295388]
[0.7420646447256184, 0.8942622950819672]
[0.8956401190012275, 0.8798116636181713]
